In [ ]:
# Reload modules automatically
%load_ext autoreload
%autoreload 2

# Simple Soil Profile

`simplesoilprofile` (`ssp`) is a lightweight library for creating and working with soil profiles in hydrological modelling workflows. It helps to initially conceptualize profiles, discretize and parametrize them. `ssp` also integrates with `dovwms`, an API wrapper for Databank Ondergrond Vlaanderen.

This notebook demonstrates a complete workflow:

- Constructing a conceptual soil profile from individual `SoilLayer` objects.
- Configuring numerical discretization for each layer (even / logarithmic splits).
- Visualizing the profile and its sublayer boundaries with `plot_profile`.
- Converting the profile to model-ready tables used by SWAP / pySWAP (for example `SOILHYDRFUNC` and `SOILPROFILE`).

Run the cells below to recreate the example profile, inspect discretization patterns, and generate the export tables that downstream models expect.

In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import Point

from simplesoilprofile import SoilLayer, SoilProfile, plot_profile

## Layer
The basic building block of a profile is a Layer. It's understood here as a portion of a profile having specific physical and hydraulic properties. Layers also hold information about numerical discratization. We will create a simple profile with 3 layers.

In [ ]:
# Create soil layers
topsoil = SoilLayer(
    name="Topsoil",
    theta_res=0.02,
    theta_sat=0.4,
    alpha=0.02,
    n=1.5,
    k_sat=10.0,
    texture_class="sandy loam",
)

subsoil = SoilLayer(
    name="Subsoil",
    theta_res=0.05,
    theta_sat=0.45,
    alpha=0.01,
    n=1.3,
    k_sat=5.0,
    texture_class="clay loam",
)

bedrock = SoilLayer(
    name="Bedrock",
    theta_res=0.1,
    theta_sat=0.3,
    alpha=0.005,
    n=1.2,
    k_sat=1.0,
    texture_class="rock",
)

# Create soil profile
profile = SoilProfile(
    name="Test Profile",
    layers=[topsoil, subsoil, bedrock],
    layer_bottoms=[30, 100, 200],
    location=Point(247172.56, 204590.58),
    elevation=5.0,
)

# Create plot
fig, ax = plt.subplots(figsize=(4, 6))
plot_profile(profile, ax=ax, show_layer_properties=True)
plt.show()

### Layer discretization
In most models, the conceptual model of the soil column is further divided into compartments between which fluxes are calculated. simplesoilprofile provides several ways to do the discratization automatically. Below we are going to define three LayerDiscretization objects. They are simple objects merely holding the information about the type and parameters of the discretization to be applied to the layer. There are three currently available types:

- Even split - user provides the number of compartments needed and the the H of each compartment is adjusted to fit the required number of sublayers.
- logarythmic split (fine top) - user provide the required number of layer and log density, which when applied creates compartments H logarythmically increasing downwards.
- logarythmic split (fine bottom) - the same as above, but reversed.
- logarythmic split symetrical - the split is done both ways looking from the center. The middle compartments are the biggest and the H of compartments decreases towards the boundaries of the layers.

After creating the discretization configuration and adding it them to the layers we can plot the new profile which now includes the discretization.

In [ ]:
from simplesoilprofile.models.discretization import DiscretizationType, LayerDiscretization

even = LayerDiscretization(
        type=DiscretizationType.EVEN,
        num_sublayers=5,
        num_compartments=10,
    )

log_both = LayerDiscretization(
        type=DiscretizationType.LOG_BOTH,
        num_sublayers=11,
        num_compartments=5,
        log_density=3
    )
log_top = LayerDiscretization(
        type=DiscretizationType.LOG_TOP,
        num_sublayers=8,
        num_compartments=2,
        log_density=2.0
    )


In [ ]:
# Add the discretizations to the layers
topsoil.discretization, subsoil.discretization, bedrock.discretization = even, log_top, log_both

In [ ]:
bedrock.discretization.compartment_heights

In [ ]:
fig, ax = plt.subplots(figsize=(8, 12))
plot_profile(profile, ax=ax, show_layer_properties=True, show_sublayers=True)
plt.show()

## Convert to pySWAP tables
Now we can convert the SoilProfile object to the tables that are needed in pyswap (and SWAP, eventually)

In [ ]:
from simplesoilprofile.models.swap import (
    profile_to_soilhydfunc_table,
    profile_to_sublayer_table,
    profile_to_texture_table,
)

# Convert profile to SWAP tables
soil_hydraulic = profile_to_soilhydfunc_table(profile)

# Display the hydraulic parameters table
print("SOILHYDRFUNC table:")
print("==================")
print(soil_hydraulic.round(4))

soil_profile = profile_to_sublayer_table(profile)

# Display the soil profile table
print("\nSOILPROFILE table:")
print("==================")
print(soil_profile.round(4))


soil_texture = profile_to_texture_table(profile)
# Display the soil texture table
print("\nSOILTEXTURE table:")
print("==================")
print(soil_texture.round(4))


# Optional: Save to CSV files
# soil_hydraulic.to_csv('soil_hydraulic.csv', index=False)
# soil_profile.to_csv('soil_profile.csv', index=False)

In [ ]:
profile.layers[2].discretization.compartment_heights

# Get data from DOV

In [ ]:
import simplesoilprofile as ssp

profile_modelled = ssp.get_profile_from_dov(profile.location)

In [ ]:
from simplesoilprofile.models.discretization import DiscretizationType, LayerDiscretization

modelled_disc_even = LayerDiscretization(
    type=DiscretizationType.EVEN,
    num_sublayers=5,
    num_compartments=10
)

for layer in profile_modelled.layers:
    layer.discretization = modelled_disc_even

In [ ]:
fig, ax = plt.subplots(figsize=(8, 12))
plot_profile(profile_modelled, ax=ax, show_layer_properties=True, show_sublayers=True)
plt.show()

In [ ]:
from simplesoilprofile.models.swap import (
    profile_to_soilhydfunc_table,
    profile_to_sublayer_table,
    profile_to_texture_table,
)

# Convert profile to SWAP tables
modelled_soil_hydraulic = profile_to_soilhydfunc_table(profile_modelled)

# Display the hydraulic parameters table
print("SOILHYDRFUNC table:")
print("==================")
print(modelled_soil_hydraulic.round(4))

modelled_soil_profile = profile_to_sublayer_table(profile_modelled)

# Display the soil profile table
print("\nSOILPROFILE table:")
print("==================")
print(modelled_soil_profile.round(4))


modelled_soil_texture = profile_to_texture_table(profile_modelled)
# Display the soil texture table
print("\nSOILTEXTURE table:")
print("==================")
print(modelled_soil_texture.round(4))

## Compute the properties with Rosetta



In [ ]:
profile_modelled

## Get texture info from texture class

In [ ]:
from simplesoilprofile.models.profile import get_profile_from_dov

profile = get_profile_from_dov(location=Point(247172.56, 204590.58), fetch_elevation=True)
